In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
#BaggingClassifier
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [2]:
import warnings

In [3]:
data=pd.read_csv('internships.csv')
data.head()

,Company,Job Title,Location,Job Type,Experience,Skill 1,Skill 2,Skill 3,Skill 4,Skill 5,Skill 6,Perk 1,Perk 2,Perk 3
0,Zscaler,Data Science Intern- Undergrad (Summer 2023),"San Jose, CA, United States",Internship,Entry-level,Agile,Architecture,Data analysis,Data Analytics,Data strategy,Data visualization,Competitive pay,Equity,Health care
1,RYTE Corporation,R&D Data Scientist Intern (Alternance),"Nice, Provence-Alpes-Côte d'Azur, France",Internship,Entry-level,Big Data,Computer Science,Deep Learning,Engineering,Keras,Python,Career development,Flex vacation,NaN
2,SOMFY Group,Alternance Data Analyst (H/F),"Cluses, France",Full Time,Entry-level,Data analysis,Excel,Power BI,Python,R,R&D,Salary bonus,NaN,NaN
3,RYTE Corporation,MLOps Engineer Intern (Alternance),"Nice, Provence-Alpes-Côte d'Azur, France",Internship,Entry-level,Azure,Big Data,CI/CD,Computer Science,Databricks,Deep Learning,Career development,NaN,NaN
4,RYTE Corporation,Data Scientist (NLP & ML) Intern (Alternance),"Nice, Provence-Alpes-Côte d'Azur, France",Internship,Entry-level,Azure,Big Data,C++,Computer Science,Deep Learning,Docker,Career development,NaN,NaN


In [4]:
data.isnull().sum()

Company         0
Job Title       0
Location        0
Job Type        0
Experience      0
Skill 1         0
Skill 2         2
Skill 3         6
Skill 4         9
Skill 5        15
Skill 6        24
Perk 1         37
Perk 2        101
Perk 3        141
dtype: int64

In [5]:
data1=data.drop(['Perk 3','Perk 2','Perk 1'],axis=1)
data1.head()

,Company,Job Title,Location,Job Type,Experience,Skill 1,Skill 2,Skill 3,Skill 4,Skill 5,Skill 6
0,Zscaler,Data Science Intern- Undergrad (Summer 2023),"San Jose, CA, United States",Internship,Entry-level,Agile,Architecture,Data analysis,Data Analytics,Data strategy,Data visualization
1,RYTE Corporation,R&D Data Scientist Intern (Alternance),"Nice, Provence-Alpes-Côte d'Azur, France",Internship,Entry-level,Big Data,Computer Science,Deep Learning,Engineering,Keras,Python
2,SOMFY Group,Alternance Data Analyst (H/F),"Cluses, France",Full Time,Entry-level,Data analysis,Excel,Power BI,Python,R,R&D
3,RYTE Corporation,MLOps Engineer Intern (Alternance),"Nice, Provence-Alpes-Côte d'Azur, France",Internship,Entry-level,Azure,Big Data,CI/CD,Computer Science,Databricks,Deep Learning
4,RYTE Corporation,Data Scientist (NLP & ML) Intern (Alternance),"Nice, Provence-Alpes-Côte d'Azur, France",Internship,Entry-level,Azure,Big Data,C++,Computer Science,Deep Learning,Docker


In [6]:
data1.isnull().sum() , data1.isnull().sum().sum()

(Company        0
 Job Title      0
 Location       0
 Job Type       0
 Experience     0
 Skill 1        0
 Skill 2        2
 Skill 3        6
 Skill 4        9
 Skill 5       15
 Skill 6       24
 dtype: int64,
 np.int64(56))

In [7]:
data1['Skill 2']=data1['Skill 2'].apply(lambda x: x.lower() if pd.notna(x) else 'none')
data1['Skill 3']=data1['Skill 3'].apply(lambda x: x.lower() if pd.notna(x) else 'none')
data1['Skill 4']=data1['Skill 4'].apply(lambda x: x.lower() if pd.notna(x) else 'none')
data1['Skill 5']=data1['Skill 5'].apply(lambda x: x.lower() if pd.notna(x) else 'none')
data1['Skill 6']=data1['Skill 6'].apply(lambda x: x.lower() if pd.notna(x) else 'none')

In [8]:
data1.isnull().sum() , data1.isnull().sum().sum()

(Company       0
 Job Title     0
 Location      0
 Job Type      0
 Experience    0
 Skill 1       0
 Skill 2       0
 Skill 3       0
 Skill 4       0
 Skill 5       0
 Skill 6       0
 dtype: int64,
 np.int64(0))

In [9]:
data1['Job Title']=data1['Job Title'].apply(lambda x: x.lower())
data1['Location']=data1['Location'].apply(lambda x: x.lower())
data1['Company']=data1['Company'].apply(lambda x: x.lower())
data1['Job Type']=data1['Job Type'].apply(lambda x: x.lower())
data1['Experience']=data1['Experience'].apply(lambda x: x.lower())
data1['Skill 1']=data1['Skill 1'].apply(lambda x: x.lower())

In [10]:
data1.groupby('Job Title')['Job Title'].count().sort_values(ascending=False)

Job Title
data science intern                                                                       11
[intern] ai/ computer vision engineer                                                      3
data analyst intern                                                                        3
data scientist intern                                                                      3
data engineer / data analyst - stage de fin d'études - paris 2e                            3
                                                                                          ..
summer internship, product management | data management (us)                               1
summer internship, research scientist, audio intelligence | music mission (emea or us)     1
website & data analyses - internship eu student                                            1
📈 nlp research intern                                                                      1
📊 data analyst intern - business & product ops              

In [11]:
data1.groupby('Company')['Company'].count().sort_values(ascending=False)

Company
bosch group         21
publicis groupe      8
cleverconnect        7
ryte corporation     6
spotify              6
                    ..
voodoo               1
wisk aero            1
zeta global          1
zscaler              1
zuora                1
Name: Company, Length: 123, dtype: int64

In [12]:
data1.head()

,Company,Job Title,Location,Job Type,Experience,Skill 1,Skill 2,Skill 3,Skill 4,Skill 5,Skill 6
0,zscaler,data science intern- undergrad (summer 2023),"san jose, ca, united states",internship,entry-level,agile,architecture,data analysis,data analytics,data strategy,data visualization
1,ryte corporation,r&d data scientist intern (alternance),"nice, provence-alpes-côte d'azur, france",internship,entry-level,big data,computer science,deep learning,engineering,keras,python
2,somfy group,alternance data analyst (h/f),"cluses, france",full time,entry-level,data analysis,excel,power bi,python,r,r&d
3,ryte corporation,mlops engineer intern (alternance),"nice, provence-alpes-côte d'azur, france",internship,entry-level,azure,big data,ci/cd,computer science,databricks,deep learning
4,ryte corporation,data scientist (nlp & ml) intern (alternance),"nice, provence-alpes-côte d'azur, france",internship,entry-level,azure,big data,c++,computer science,deep learning,docker


In [13]:
l=LabelEncoder()

In [14]:
data2=data1

In [15]:
data2['Skill 1']=l.fit_transform(data2['Skill 1'])
data2['Skill 2']=l.fit_transform(data2['Skill 2'])
data2['Skill 3']=l.fit_transform(data2['Skill 3'])
data2['Skill 4']=l.fit_transform(data2['Skill 4'])
data2['Skill 5']=l.fit_transform(data2['Skill 5'])
data2['Skill 6']=l.fit_transform(data2['Skill 6'])
data2['Job Title']=l.fit_transform(data2['Job Title'])
data2['Location']=l.fit_transform(data2['Location'])
data2['Company']=l.fit_transform(data2['Company'])
data2['Job Type']=l.fit_transform(data2['Job Type'])
data2['Experience']=l.fit_transform(data2['Experience'])

In [16]:
data2.head()

,Company,Job Title,Location,Job Type,Experience,Skill 1,Skill 2,Skill 3,Skill 4,Skill 5,Skill 6
0,121,90,108,1,0,1,4,18,12,11,8
1,82,163,76,1,0,17,19,28,29,40,62
2,95,16,31,0,0,34,41,56,64,66,64
3,82,151,76,1,0,13,8,12,7,13,12
4,82,95,76,1,0,13,8,8,7,15,15


In [17]:
feature=data2.drop(['Job Title'],axis=1)
feature.head()

,Company,Location,Job Type,Experience,Skill 1,Skill 2,Skill 3,Skill 4,Skill 5,Skill 6
0,121,108,1,0,1,4,18,12,11,8
1,82,76,1,0,17,19,28,29,40,62
2,95,31,0,0,34,41,56,64,66,64
3,82,76,1,0,13,8,12,7,13,12
4,82,76,1,0,13,8,8,7,15,15


In [18]:
target=pd.DataFrame()
target['Job Title']=data2['Job Title']
target

,Job Title
0,90
1,163
2,16
3,151
4,95
...,...
223,72
224,145
225,20
226,182


## using ShuffleSplit (not good for small data)

In [19]:
cv=ShuffleSplit(n_splits=2)

In [20]:
model1=LogisticRegression(max_iter=500)
warnings.filterwarnings('ignore')
cross_val_score(model1,feature,target,cv=cv)

array([0.13043478, 0.08695652])

In [21]:
model2=GaussianNB()
warnings.filterwarnings('ignore')
cross_val_score(model2,feature,target,cv=cv)

array([0.17391304, 0.        ])

In [22]:
model3=DecisionTreeClassifier()
cross_val_score(model3,feature,target,cv=cv)

array([0.08695652, 0.13043478])

In [23]:
model4=RandomForestClassifier(n_estimators=200)
warnings.filterwarnings('ignore')
cross_val_score(model4,feature,target,cv=cv)

array([0.17391304, 0.08695652])

In [24]:
model5=KNeighborsClassifier(n_neighbors=5)
warnings.filterwarnings('ignore')
cross_val_score(model5,feature,target,cv=cv)

array([0., 0.])

In [25]:
model6=BaggingClassifier(n_estimators=100,estimator=model1)
warnings.filterwarnings('ignore')
m1=cross_val_score(model6,feature,target,cv=cv)
warnings.filterwarnings('ignore')
m1

array([0.08695652, 0.08695652])

In [26]:
model7=BaggingClassifier(n_estimators=100,estimator=model2)
warnings.filterwarnings('ignore')
m2=cross_val_score(model7,feature,target,cv=cv)
warnings.filterwarnings('ignore')
m2

array([0.08695652, 0.        ])

In [27]:
model8=BaggingClassifier(n_estimators=100,estimator=model3)
warnings.filterwarnings('ignore')
m3=cross_val_score(model8,feature,target,cv=cv)
warnings.filterwarnings('ignore')
m3

array([0.2173913 , 0.08695652])

In [28]:
model9=BaggingClassifier(n_estimators=100,estimator=model4)
warnings.filterwarnings('ignore')
m4=cross_val_score(model9,feature,target,cv=cv)
warnings.filterwarnings('ignore')
m4

array([0.13043478, 0.17391304])

In [29]:
model10=BaggingClassifier(n_estimators=100,estimator=model5)
warnings.filterwarnings('ignore')
m5=cross_val_score(model10,feature,target,cv=cv)
warnings.filterwarnings('ignore')
m5

array([0., 0.])

## using train_test_split 

In [30]:
x,xt,y,yt=train_test_split(feature,target,test_size=0.2)

In [31]:
model1=LogisticRegression(max_iter=500)
model1.fit(x,y)
warnings.filterwarnings('ignore')
model1.score(xt,yt)

0.13043478260869565

In [32]:
model2=GaussianNB()
model2.fit(x,y)
warnings.filterwarnings('ignore')
model2.score(xt,yt)

0.15217391304347827

In [33]:
model3=DecisionTreeClassifier()
model3.fit(x,y)
warnings.filterwarnings('ignore')
model3.score(xt,yt)

0.13043478260869565

In [34]:
model4=RandomForestClassifier(n_estimators=200)
model4.fit(x,y)
warnings.filterwarnings('ignore')
model4.score(xt,yt)

0.13043478260869565

In [35]:
model5=KNeighborsClassifier(n_neighbors=5)
model5.fit(x,y)
warnings.filterwarnings('ignore')
model5.score(xt,yt)

0.021739130434782608

In [36]:
model6=BaggingClassifier(n_estimators=100,estimator=model1)
model6.fit(x,y)
warnings.filterwarnings('ignore')
model6.score(xt,yt)

0.13043478260869565

In [37]:
model7=BaggingClassifier(n_estimators=100,estimator=model2)
model7.fit(x,y)
warnings.filterwarnings('ignore')
model7.score(xt,yt)

0.13043478260869565

In [38]:
model8=BaggingClassifier(n_estimators=100,estimator=model3)
model8.fit(x,y)
warnings.filterwarnings('ignore')
model8.score(xt,yt)

0.13043478260869565

In [39]:
model9=BaggingClassifier(n_estimators=100,estimator=model4)
model9.fit(x,y)
warnings.filterwarnings('ignore')
model9.score(xt,yt)

0.13043478260869565

In [40]:
model10=BaggingClassifier(n_estimators=100,estimator=model5)
model10.fit(x,y)
warnings.filterwarnings('ignore')
model10.score(xt,yt)

0.08695652173913043

In [41]:
# i was planning to make some plots, but since the results are too weak, I didn’t go ahead with it.